In [6]:
from galapy.PhotometricSystem import print_filters
print_filters()


# Path: /home/polaris/.galapy

l68
347660922309852
HST.WFPC2.WF.F547M
HST.WFC3.UVIS2.F275W
HST.WFC3.UVIS2.F350LP
HST.WFC3.UVIS2.F475X
HST.WFC3.UVIS2.F390W
HST.WFC3.UVIS2.F225W
HST.WFC3.UVIS2.F606W
HST.WFC3.UVIS2.F600LP
HST.WFC3.UVIS2.F336W
HST.WFC3.IR.F105W
HST.WFC3.IR.F110W
HST.WFC3.IR.F098M
HST.WFC3.IR.F160W
HST.ACS.WFC.F625W
JCMT.SCUBA.2450MHz
JCMT.SCUBA.2850MHz
CFHT.MegaCam.u_1
ATCA.ATCA2
ATCA.ATCA1
GOODS.z
GOODS.b
GOODS.v
GOODS.i
UKIDSS.Y
WISE.W3
WISE.W4
WISE.W2
WISE.W1
2MASS.H
2MASS.J
2MASS.Ks
ALMA.B8
ALMA.B6
ALMA.B7
ALMA.B3
GALEX.FUV
GALEX.NUV
Subaru.HSC.z_filter
Subaru.HSC.i_filter
Subaru.HSC.r_filter
Subaru.Suprime.B_filter
Subaru.Suprime.V_filter
Spitzer.IRAC.I3
Spitzer.IRAC.I2
Spitzer.IRAC.I1
Spitzer.IRAC.I4
Spitzer.MIPS.160mu
Spitzer.MIPS.24mu
Spitzer.MIPS.70mu
Spitzer.IRS.Red
Spitzer.IRS.Blue
JWST.NIRISS.F115W
JWST.NIRISS.F150W
JWST.NIRISS.F430M
JWST.NIRISS.F480M
JWST.NIRISS.F356W
JWST.NIRISS.F140M
JWST.NIRISS.F090W
JWST.NIRISS.F444W
JWST.NIRISS.F200W
JWST.NIRISS.F277W
JW

In [ ]:
# Filter the multiwavelenght catalog to keep only the columns needed
emu_columns = [ 'EMU_ra_deg_cont', 'EMU_dec_deg_cont']

# desdr2_mag = ['DES_mag_auto_g_dered', 'DES_mag_auto_r_dered', 'DES_mag_auto_i_dered', 'DES_mag_auto_z_dered']
# desdr2_colors = ['DES_g_r_dered', 'DES_r_i_dered', 'DES_i_z_dered']

# Technically only need one magnitude to recover the rest from the colors
desy6gold_mag = ['DESY6_mag_auto_g_extcorr', 'DESY6_mag_auto_r_extcorr', 'DESY6_mag_auto_i_extcorr', 'DESY6_mag_auto_z_extcorr', 'DESY6_mag_auto_y_extcorr']
desy6gold_colors = ['DESY6_g_r_extcorr', 'DESY6_r_i_extcorr', 'DESY6_i_z_extcorr', 'DESY6_z_y_extcorr']
desy6gold_features = ['DESY6_dnf_z', 'DESY6_spread_model_g', 'DESY6_spread_model_r', 'DESY6_spread_model_i', 'DESY6_spread_model_z']

viking_mag = ['VKG_zAperMag3_ab_extcorr', 'VKG_jAperMag3_ab_extcorr', 'VKG_yAperMag3_ab_extcorr', 'VKG_ksAperMag3_ab_extcorr', 'VKG_hAperMag3_ab_extcorr']
viking_colors = ['VKG_z_y_am3_extcorr', 'VKG_y_j_am3_extcorr', 'VKG_j_h_am3_extcorr', 'VKG_h_ks_am3_extcorr']
viking_features = ['VKG_mergedClassStat']

catwise_mag = ['CAT_w1mpro_ab', 'CAT_w2mpro_ab']
catwise_colors = ['CAT_w1_w2_ab']

colors = desy6gold_colors + viking_colors + catwise_colors

## Quantization Error (QE) and Topographic Error (TE)

In [ ]:
# Quantization Error (QE)
# Average distance of a data point to the nearest lattice node
# Measures how well the mapping fits the distribution of the data

def som_quantization_error(data, trained_som):
    '''
    Calculates the quantization error of a trained SOMNet object

    Args:
        data: NumPy array of input data.
        trainded_som: trained SimpSOM SOMNet object

    Returns:
        quantization error (float)
    '''
    # Convert to clean NumPy array with uniform dtype first
    data_np = np.asarray(data, dtype=np.float64)

    # Convert NumPy array to CuPy array for GPU processing
    data_cp = cp.array(data_np)

    # Find all BMU indices
    bmu_indices = trained_som.find_bmu_ix(data_cp)

    # Get the weights of the BMUs
    bmu_weights = cp.array([trained_som.nodes_list[int(bmu_idx)].weights for bmu_idx in bmu_indices])

    # Vectorized distance calculation
    distances = cp.linalg.norm(bmu_weights - data_cp, axis=1)
    total_distance = cp.sum(distances)

    quantization_error = float(total_distance / len(data))
    return quantization_error

In [ ]:
# Topographic Error (TE)
# Proportion of data points whose BMU and second BMU are NOT neighbors
# Measures how well the shape of the data is preserved in the output space

def som_topographic_error(data, trained_som):
    """
    Computes topographic error ET for a SOM with hex topology using GPU (CuPy).
    
    Args:
        data:    The input data used to train the SOM (NumPy array).
        trained_som: A trained SOMNet object from the simpsom library.

    Returns:
        Topographic error (float between 0 and 1).
        Proportion of data points whose BMU and second BMU are NOT neighbors
    """
    # Convert data to CuPy arrays
    data_cp = cp.array(data)

    # Get all the nodes' weights from the trained SOM (GPU-enabled)
    weights_cp = cp.array([node.weights for node in trained_som.nodes_list])

    # Initialize total error
    total_error = 0

    # Find all BMU indices
    bmu_indices = trained_som.find_bmu_ix(data_cp)

    # Find all the second BMU indices
    som_dist = sps.distances.Distance(xp=np) # Initialize the distance object, as required by the documentation
    distances = som_dist.pairdist(data_cp, weights_cp, metric='euclidean')

    sorted_indices = cp.argsort(distances, axis=1)
    sbmu_indices = sorted_indices[:, 1]


    # Get the positions of the BMU and second BMU in the grid
    bmu_positions = cp.array([trained_som.nodes_list[int(bmu_idx)].pos for bmu_idx in bmu_indices])
    sbmu_positions = cp.array([trained_som.nodes_list[int(sbmu_idx)].pos for sbmu_idx in sbmu_indices])


    bmu_row, bmu_col = bmu_positions[:, 0], bmu_positions[:, 1]
    sbmu_row, sbmu_col = sbmu_positions[:, 0], sbmu_positions[:, 1]

    # Check if BMU and sBMU are neighbors
    row_neighbors = cp.abs(bmu_row - sbmu_row)
    col_neighbors = cp.abs(bmu_col - sbmu_col)

    not_neighbors = (row_neighbors > 1) | (col_neighbors > 1)

    number_not_neighbors = cp.sum(not_neighbors)

    # Compute the topographic error
    topographic_error = number_not_neighbors / data_cp.shape[0]

    return float(topographic_error)

# Add custom DES filters

In [ ]:
g_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.g.dat'
i_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.i.dat'
r_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.r.dat'
z_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.z.dat'
y_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.Y.dat'

In [ ]:
def normalize_filter_data(input_filename, output_filename):
    """
    Normalizes the second column (transmission values) of a .dat file.

    Parameters:
    - input_filename (str): Path to the input .dat file.
    - output_filename (str): Path where the normalized data will be saved.

    The input file should contain two columns: wavelength (first column) and transmission (second column).
    """
    # Initialize lists to store data
    lambda_values = []
    transmission_values = []

    # Read the data from the .dat file
    with open(input_filename, 'r') as f:
        for line in f:
            # Skip comments or empty lines
            if line.startswith('#') or line.strip() == '':
                continue
            
            # Split the line into wavelength and transmission
            columns = line.split()
            lambda_values.append(float(columns[0]))  # Wavelength (first column)
            transmission_values.append(float(columns[1]))  # Transmission (second column)

    # Convert the lists to numpy arrays for easier processing
    lambda_values = np.array(lambda_values)
    transmission_values = np.array(transmission_values)

    # Normalize the transmission values (second column)
    max_transmission = np.max(transmission_values)
    normalized_transmission = transmission_values / max_transmission

    # Save the normalized data to the output .dat file
    with open(output_filename, 'w') as f:
        # Optionally add a header
        f.write("# Wavelength (Å) and Normalized Transmission Values\n")
        
        # Write the wavelength and the normalized transmission
        for lambda_val, norm_trans in zip(lambda_values, normalized_transmission):
            f.write(f"{lambda_val} {norm_trans}\n")

    print(f"Normalization complete. Output saved to {output_filename}.")

In [ ]:
# normalize_filter_data(g_filter_path, g_filter_path)
# normalize_filter_data(i_filter_path, i_filter_path)
# normalize_filter_data(r_filter_path, r_filter_path)
# normalize_filter_data(y_filter_path, y_filter_path)
# normalize_filter_data(z_filter_path, z_filter_path)